# Final exam

**For this exam, feel free to re-use any code from the previous lab notebooks.**

#### Tasks
- Use accelerator data to construct a neural network surrogate model, train that model, and demonstrate that it accurately models the data
- Use Bayesian optimization to optimize the function and determine the best operational parameters

## Setting up the environment

In [ ]:
!pip install --upgrade git+https://github.com/uspas/2021_optimization_and_ml --quiet

In [ ]:
%reset -f

import numpy as np
import matplotlib.pyplot as plt

#matplotlib graphs will be included in your notebook, next to the code:
%matplotlib inline

#add PyTorch and TorchVision (used for cropping etc.)
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#import toy accelerator package
from uspas_ml.accelerator_toy_models import simple_linarc
from uspas_ml.utils import transformer

## Generate dataset

The code below generates samples from "simulations" of a beam propagating through a linac followed by an arc:
- The input to the simulator are the linac's phase, the arc's R56 coefficient, and the beam final (mean) energy.
- The output of the simulator are the bunch length (in meters) and the bunch's RMS energy spread.

In [ ]:
#generate input samples
n = 10000
torch.manual_seed(0)
inputs = torch.rand((n, 3))

#phase +/- 50 deg
inputs[:, 0] = inputs[:, 0] * 100.0 - 50.0

#r56 +/- 0.5
inputs[:,1] = inputs[:,1] - 0.5

#final energy [800,1300]*1e6
inputs[:,2] = (inputs[:,2] * 500.0 + 800.0) * 1e6

print('Inputs:')
print(inputs)

outputs = []
for i in range(n):
    outputs += [simple_linarc.propagate(linac_phase = inputs[i][0], 
                                        arc_r56 = inputs[i][1],
                                        linac_final_E = inputs[i][2])]

outputs = torch.vstack(outputs)

print('Outputs:')
print(outputs)

## Regression with a neural network

<div class="alert alert-block alert-info">
    
**Task:** 
    Appropriately normalize the input data, and standardize the output data.
</div>

<div class="alert alert-block alert-info">
    
**Task:** 
    Create and train a neural network to model this data, i.e. for each data point, the neural network should take the above 3 (normalized) input features and predict the above 2 (normalized) output features. 
    
In order to show that the neural network works as expected and that the training roughly converged, plot the evolution of the loss function - both for the training dataset and test dataset - during training. (Use the first 7000 data points as the training set, and the remaining 3000 data points as the test set.)
</div>

## Bayesian optimization

<div class="alert alert-block alert-info">
    
**Task:** 
    Use Bayesian optimization to minimize the bunch length (i.e. the first of the 2 features that are returned by `simple_linarc`) with respect to phase, r56, and final_energy (i.e. the 3 input features that we passed to `simple_linarc` when previously generating the dataset for the neural network). 
    
Use the first 10 elements of the previously-created arrays `inputs` and `outputs` as the initial dataset on which to fit the initial Gaussian Process (at the beginning of Bayesian optimization). Then, at each iteration of Bayesian optimization, call the `simple_linarc` function on the new candidate point. 
    
Run 6 steps of Bayesian optimization and print the values of the bunch length obtained at each iteration. What is the best value obtained so far?
</div>